# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,12.68,98,100,3.13,NZ,1723931903
1,1,cape coast,5.1053,-1.2466,23.36,87,91,3.75,GH,1723931904
2,2,bothaville,-27.3887,26.6170,11.62,17,0,1.93,ZA,1723931905
3,3,carnarvon,-24.8667,113.6333,18.11,57,97,6.00,AU,1723931907
4,4,khandyga,62.6667,135.6000,10.62,84,13,0.82,RU,1723931908


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",                     
    "Lat",                   
    geo=True,                   
    tiles="OSM",                 
    frame_width=700,            
    frame_height=500,            
    size="Humidity",             
    scale=1,                  
    color="City",               
)
# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
cities_df = city_data_df[
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_cities_df = cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
83,83,prince george,53.9166,-122.7530,22.01,54,0,1.54,CA,1723931791
219,219,palestine,31.9216,35.2033,24.94,58,0,1.79,PS,1723932188
330,330,mont-dore,-22.2833,166.5833,23.89,99,0,0.78,NC,1723932323
355,355,bani walid,31.7566,13.9942,26.40,53,0,3.26,LY,1723932355
398,398,buritis,-15.6178,-46.4233,26.05,26,0,1.98,BR,1723932410
429,429,steinbach,50.1667,8.5667,21.85,94,0,0.51,DE,1723932452
455,455,olinda,-8.0089,-34.8553,25.85,65,0,4.12,BR,1723932484
471,471,pointe-noire,-4.7761,11.8635,21.64,90,0,3.13,CG,1723932505
491,491,ribeira brava,32.6500,-17.0667,24.51,52,0,1.91,PT,1723932529
535,535,nova vicosa,-17.8919,-39.3719,21.59,75,0,3.67,BR,1723932584


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
83,prince george,CA,53.9166,-122.7530,54,
219,palestine,PS,31.9216,35.2033,58,
330,mont-dore,NC,-22.2833,166.5833,99,
355,bani walid,LY,31.7566,13.9942,53,
398,buritis,BR,-15.6178,-46.4233,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set search parameters
limit = 1
radius = 5000  

# Define the base API parameters
params = {
    "categories": "accommodation.hotel",
    "limit": limit,
    "apiKey": geoapify_key  # Replace with your actual API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Update the filter and bias for the current city
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
prince george - nearest hotel: Travelodge by Wyndham Prince George
palestine - nearest hotel: Retno Hotel
mont-dore - nearest hotel: No hotel found
bani walid - nearest hotel: فندق الزيتونة
buritis - nearest hotel: No hotel found
steinbach - nearest hotel: Alt Steinbach
olinda - nearest hotel: Fidji Motel
pointe-noire - nearest hotel: HÔTEL F.A.R.E
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
nova vicosa - nearest hotel: Pousada Pontal da Barra
santa rita do passa quatro - nearest hotel: Hotel Pousada da Colina
nazilli - nearest hotel: No hotel found
pompeia - nearest hotel: Hotel Pousada Vale do Sol
es castell - nearest hotel: Hotel Agamenon


,City,Country,Lat,Lng,Humidity,Hotel Name
83,prince george,CA,53.9166,-122.7530,54,Travelodge by Wyndham Prince George
219,palestine,PS,31.9216,35.2033,58,Retno Hotel
330,mont-dore,NC,-22.2833,166.5833,99,No hotel found
355,bani walid,LY,31.7566,13.9942,53,فندق الزيتونة
398,buritis,BR,-15.6178,-46.4233,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",                     
    "Lat",                   
    geo=True,                   
    tiles="OSM",                 
    frame_width=700,            
    frame_height=500,            
    size="Humidity",             
    scale=1,                  
    color="City",
    hover_cols=["Hotel Name", "Humidity", "Country"]  # Columns to display on hover
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)